In [3]:
# Install the litellm library - this is a tool that helps us talk to different AI models
!!pip install litellm

# IMPORTANT SETUP INSTRUCTIONS:
# You need to set your OpenAI API key as a secret in Google Colab
# Click the "key" icon on the left sidebar and add 'OPENAI_API_KEY' with your actual API key

# Import necessary libraries (think of these as toolboxes we need)
import os  # For working with the operating system and environment variables
from google.colab import userdata  # To get secret keys from Google Colab
from google.colab import files  # For file upload functionality in Google Colab
import json  # For working with JSON data format (a way to structure information)
import shutil  # For file operations like moving and copying files
from typing import List  # For specifying what type of data functions should return
from litellm import completion  # The main tool to communicate with AI models

# Get the OpenAI API key from Google Colab's secret storage
api_key = userdata.get('OPENAI_API_KEY')
# Set this key as an environment variable so the AI can use it
os.environ['OPENAI_API_KEY'] = api_key

# ==================== PART 1: TOOL FUNCTIONS AND DEFINITION ====================
# These are the "tools" or "skills" we're giving to our AI agent

def list_files() -> List[str]:
    """This function lists all files in the current directory (folder)."""
    return os.listdir(".")  # Get all files in the current folder and return them as a list

def read_file(file_name: str) -> str:
    """This function reads the contents of a specific file."""
    try:  # Try to do something, but be ready to handle errors
        with open(file_name, "r") as file:  # Open the file in read mode
            return file.read()  # Read all content from the file and return it
    except FileNotFoundError:  # If the file doesn't exist
        return f"Error: {file_name} not found."
    except Exception as e:  # If any other error happens
        return f"Error: {str(e)}"

def add_file(file_name: str = None, content: str = None, directory: str = "sample_data", upload_from_computer: bool = False) -> str:
    """
    This function either:
    1. Creates a new file with specified content in a given directory, OR
    2. Opens a file upload dialog for the user to select and upload a file from their computer
    """
    try:
        # If user wants to upload from computer
        if upload_from_computer:
            print("Opening file upload dialog...")
            print("Please select the file(s) you want to upload from your computer.")

            # Open the file upload dialog
            uploaded = files.upload()

            # Process each uploaded file
            results = []
            for filename, file_data in uploaded.items():
                # Create the full destination path
                destination_path = os.path.join(directory, filename)

                # Create the directory if it doesn't exist
                os.makedirs(directory, exist_ok=True)

                # Move the uploaded file to the specified directory
                if directory != "sample_data":
                    # If not sample_data directory, move the file
                    shutil.move(filename, destination_path)
                    results.append(f"Successfully uploaded '{filename}' to directory '{directory}'")
                else:
                    # Check if we need to move to sample_data directory
                    if not os.path.exists("sample_data"):
                        os.makedirs("sample_data", exist_ok=True)
                    if filename != destination_path:
                        shutil.move(filename, destination_path)
                    results.append(f"Successfully uploaded '{filename}' to sample_data directory")

            return "\n".join(results)

        # Traditional file creation mode (original functionality)
        else:
            if file_name is None or content is None:
                return "Error: file_name and content are required when not uploading from computer"

            # Create the full file path by joining directory and file name
            file_path = os.path.join(directory, file_name)

            # Create the directory if it doesn't exist
            os.makedirs(directory, exist_ok=True)

            # Write the content to the file
            with open(file_path, "w") as file:
                file.write(content)

            return f"Successfully created file '{file_name}' in directory '{directory}'"

    except PermissionError:
        return f"Error: Permission denied. Cannot write to directory '{directory}'"
    except Exception as e:
        return f"Error: {str(e)}"

def create_documentation(file_name: str, output_directory: str = ".") -> str:
    """This function reads a code file and creates comprehensive documentation for it."""
    try:
        # First, check if the source file exists
        if not os.path.exists(file_name):
            return f"Error: Source file '{file_name}' not found."

        # Read the source code file
        with open(file_name, "r") as file:
            code_content = file.read()

        # Create documentation file name based on source file
        base_name = os.path.splitext(os.path.basename(file_name))[0]
        doc_file_name = f"{base_name}_documentation.txt"
        doc_file_path = os.path.join(output_directory, doc_file_name)

        # Create the output directory if it doesn't exist
        os.makedirs(output_directory, exist_ok=True)

        # Generate documentation content
        documentation = f"""
# CODE DOCUMENTATION FOR: {file_name}
{'=' * 60}

## FILE INFORMATION
- Source File: {file_name}
- Documentation Generated: {os.path.basename(doc_file_path)}
- File Size: {len(code_content)} characters
- Line Count: {len(code_content.splitlines())} lines

## FILE OVERVIEW
This document provides detailed documentation for the code in {file_name}.

## CODE STRUCTURE ANALYSIS

### Imports and Dependencies
"""

        # Analyze imports (works for Python, can be extended for other languages)
        lines = code_content.splitlines()
        imports = []
        functions = []
        classes = []
        comments = []

        current_function = None
        function_lines = []

        for i, line in enumerate(lines, 1):
            stripped_line = line.strip()

            # Find imports
            if stripped_line.startswith(('import ', 'from ')):
                imports.append(f"Line {i}: {stripped_line}")

            # Find function definitions
            elif stripped_line.startswith('def '):
                if current_function:
                    functions.append((current_function, function_lines))
                current_function = f"Line {i}: {stripped_line}"
                function_lines = []

            # Find class definitions
            elif stripped_line.startswith('class '):
                classes.append(f"Line {i}: {stripped_line}")

            # Find comments and docstrings
            elif stripped_line.startswith('#') or '"""' in stripped_line or "'''" in stripped_line:
                comments.append(f"Line {i}: {stripped_line}")

            # Collect function body
            if current_function and not stripped_line.startswith('def '):
                function_lines.append(line)

        # Add last function if exists
        if current_function:
            functions.append((current_function, function_lines))

        # Add imports to documentation
        if imports:
            for imp in imports:
                documentation += f"\n{imp}"
        else:
            documentation += "\nNo imports found."

        # Add functions to documentation
        documentation += f"\n\n### Functions Found ({len(functions)} total)\n"
        if functions:
            for func_def, func_body in functions:
                documentation += f"\n{func_def}\n"
                # Try to find docstring in function body
                docstring_found = False
                for line in func_body[:5]:  # Check first 5 lines for docstring
                    if '"""' in line or "'''" in line:
                        documentation += f"  Documentation: {line.strip()}\n"
                        docstring_found = True
                        break
                if not docstring_found:
                    documentation += "  Documentation: No docstring found\n"
        else:
            documentation += "No functions found."

        # Add classes to documentation
        documentation += f"\n\n### Classes Found ({len(classes)} total)\n"
        if classes:
            for cls in classes:
                documentation += f"\n{cls}"
        else:
            documentation += "No classes found."

        # Add comments analysis
        documentation += f"\n\n### Comments and Documentation ({len(comments)} total)\n"
        if comments:
            for comment in comments[:10]:  # Show first 10 comments
                documentation += f"\n{comment}"
            if len(comments) > 10:
                documentation += f"\n... and {len(comments) - 10} more comments"
        else:
            documentation += "No comments found."

        # Add code complexity analysis
        documentation += f"\n\n### Code Complexity Analysis\n"
        documentation += f"- Total lines of code: {len(lines)}\n"
        documentation += f"- Non-empty lines: {len([l for l in lines if l.strip()])}\n"
        documentation += f"- Comment lines: {len([l for l in lines if l.strip().startswith('#')])}\n"
        documentation += f"- Function definitions: {len(functions)}\n"
        documentation += f"- Class definitions: {len(classes)}\n"
        documentation += f"- Import statements: {len(imports)}\n"

        # Add full source code at the end
        documentation += f"\n\n{'=' * 60}\n"
        documentation += f"## COMPLETE SOURCE CODE\n"
        documentation += f"{'=' * 60}\n\n"
        documentation += code_content

        # Write documentation to file
        with open(doc_file_path, "w") as doc_file:
            doc_file.write(documentation)

        return f"Successfully created documentation file '{doc_file_name}' in directory '{output_directory}'. The documentation includes code analysis, structure overview, and complete source code."

    except PermissionError:
        return f"Error: Permission denied. Cannot write to directory '{output_directory}'"
    except Exception as e:
        return f"Error creating documentation: {str(e)}"

def terminate(message: str) -> None:
    """This function ends the conversation and prints a final message."""
    print(f"Termination message: {message}")

# ==================== PART 2: TOOL SETUP ====================
# Create a dictionary that maps tool names to their actual functions
# This is like a phone book - when the AI says "call list_files", we know which function to run
tool_functions = {
    "list_files": list_files,
    "read_file": read_file,
    "add_file": add_file,
    "create_documentation": create_documentation,
    "terminate": terminate
}

# Define the tools in a format that the AI model can understand
# This is like giving the AI a manual of what each tool does and how to use it
tools = [
    {
        "type": "function",  # Tell the AI this is a function it can call
        "function": {
            "name": "list_files",  # The name of the tool
            "description": "Returns a list of files in the directory.",  # What it does
            "parameters": {"type": "object", "properties": {}, "required": []}  # No input needed
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in the directory.",
            "parameters": {
                "type": "object",
                "properties": {"file_name": {"type": "string"}},  # Needs a file name as input
                "required": ["file_name"]  # The file_name is required
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add_file",
            "description": "Creates a new file with specified content OR uploads a file from the user's computer. Set upload_from_computer=true to open file selection dialog.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {"type": "string", "description": "Name of the file to create (required only when creating new file)"},
                    "content": {"type": "string", "description": "The text content to write in the file (required only when creating new file)"},
                    "directory": {"type": "string", "description": "Directory path where to create/upload the file (default is 'sample_data' directory)"},
                    "upload_from_computer": {"type": "boolean", "description": "Set to true to open file upload dialog for selecting files from computer (default: false)"}
                },
                "required": []  # All parameters are optional depending on the mode
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_documentation",
            "description": "Analyzes a code file and creates comprehensive documentation for it in a .txt file. Includes code structure analysis, functions, classes, imports, comments, and the complete source code.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_name": {"type": "string", "description": "Path to the source code file to document (e.g., 'script.py', 'src/main.js')"},
                    "output_directory": {"type": "string", "description": "Directory where to save the documentation file (default is current directory '.')"}
                },
                "required": ["file_name"]  # output_directory is optional, defaults to current directory
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "terminate",
            "description": "Terminates the conversation. No further actions or interactions are possible after this. Prints the provided message for the user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string"},  # Needs a message as input
                },
                "required": ["message"]  # The message is required
            }
        }
    }
]

# ==================== PART 3: AI AGENT INSTRUCTIONS ====================
# These are the rules and personality we give to our AI agent
agent_rules = [{
    "role": "system",  # This is a system message (like instructions for the AI)
    "content": """
You are an AI agent that can perform tasks by using available tools.
Available tools:
- list_files: Shows all files and directories in the current location
- read_file: Reads the content of any file
- add_file: Creates new files with custom content OR uploads files from user's computer
  * To create a new file: use file_name, content, and optionally directory
  * To upload from computer: set upload_from_computer=true and optionally specify directory
- create_documentation: Analyzes code files and generates comprehensive documentation

If a user asks about files, documents, or content, first list the files before reading them.
When creating documentation, provide detailed analysis of the code structure, functions, and complexity.
When a user wants to upload files from their computer, use the add_file tool with upload_from_computer=true.

IMPORTANT: NEVER use the terminate tool unless the user explicitly tells you to "terminate" or "stop".
After completing any task, provide your results and ask what else you can help with.
Continue the conversation and wait for the user's next instruction.
Only call the terminate function if the user's message contains words like "terminate", "stop", "quit", "exit", or "done".
"""
}]

# ==================== PART 4: MAIN PROGRAM SETUP ====================
# Initialize variables to control how the agent works
iterations = 0  # Keep track of how many times we've talked to the AI
max_iterations = 10  # Maximum number of conversations to prevent infinite loops

# Ask the user what they want the AI to do
user_task = input("What would you like me to do? ")

# Create a memory list to store the conversation history
# Start with the user's request
memory = [{"role": "user", "content": user_task}]

# ==================== PART 5: THE MAIN AI AGENT LOOP ====================
# This loop keeps the conversation going until the task is done
while iterations < max_iterations:
    # Increment the iteration counter at the start of each loop
    iterations += 1

    # Combine the agent rules with the conversation memory
    messages = agent_rules + memory

    # Send everything to the AI model and get a response
    response = completion(
        model="openai/gpt-4o",  # Which AI model to use (GPT-4o from OpenAI)
        messages=messages,  # The conversation history
        tools=tools,  # The tools the AI can use
        max_tokens=1024  # Maximum length of the AI's response
    )

    # Check if the AI wants to use a tool
    if response.choices[0].message.tool_calls:
        # Extract information about which tool the AI wants to use
        tool = response.choices[0].message.tool_calls[0]  # Get the first (and only) tool call
        tool_name = tool.function.name  # What tool does it want to use?
        tool_args = json.loads(tool.function.arguments)  # What arguments/inputs for the tool?

        # Create a record of what action the AI wants to take
        action = {
            "tool_name": tool_name,
            "args": tool_args
        }

        # Handle the special "terminate" tool differently
        if tool_name == "terminate":
            print(f"Termination message: {tool_args['message']}")
            break  # Exit the loop - we're done!

        # If it's a tool we know about, execute it
        elif tool_name in tool_functions:
            try:
                # Call the actual function with the provided arguments
                # The ** syntax unpacks the arguments dictionary
                result = {"result": tool_functions[tool_name](**tool_args)}
            except Exception as e:  # If something goes wrong
                result = {"error": f"Error executing {tool_name}: {str(e)}"}

        # If the AI tried to use a tool we don't have
        else:
            result = {"error": f"Unknown tool: {tool_name}"}

        # Show what's happening (for debugging/monitoring)
        print(f"Executing: {tool_name} with args {tool_args}")
        print(f"Result: {result}")

        # Add this interaction to the conversation memory
        # First, record what the AI wanted to do
        # Then, record the result of doing it
        memory.extend([
            {"role": "assistant", "content": json.dumps(action)},
            {"role": "user", "content": json.dumps(result)}
        ])

    # If the AI doesn't want to use any tools, it's giving a regular text response
    else:
        result = response.choices[0].message.content  # Get the text response
        print(f"Response: {result}")

        # Add the AI's response to memory so it can continue the conversation
        memory.append({"role": "assistant", "content": result})

        # Ask for new user input to continue the conversation
        new_user_input = input("\nWhat else would you like me to do? (or type 'terminate' to stop): ")

        # Add the new user input to memory
        memory.append({"role": "user", "content": new_user_input})

        # Check if user wants to terminate
        if new_user_input.lower().strip() in ['terminate', 'stop', 'quit', 'exit', 'done']:
            print("Terminating as requested by user.")
            break

# ==================== END OF PROGRAM ====================
# The program ends here when either:
# 1. The AI calls the "terminate" tool
# 2. We reach the maximum number of iterations (safety measure)
#
# Note: The AI can now give text responses and continue working until it decides to terminate

What would you like me to do? list files
Executing: list_files with args {}
Result: {'result': ['.config', 'sample_data', 'AgentLoopWithFunctionCalling_Extended_documentation.txt']}
Response: Here's the list of files and directories in the current location:

1. `.config`
2. `sample_data`
3. `AgentLoopWithFunctionCalling_Extended_documentation.txt`

Is there anything specific you would like to do with these files or directories?

What else would you like me to do? (or type 'terminate' to stop): i would like to upload a file
Opening file upload dialog...
Please select the file(s) you want to upload from your computer.


Saving agent_example.py to agent_example.py
Executing: add_file with args {'upload_from_computer': True}
Result: {'result': "Successfully uploaded 'agent_example.py' to sample_data directory"}
Response: Your file `agent_example.py` has been successfully uploaded to the `sample_data` directory.

What would you like to do next? Would you like me to analyze and create documentation for this new file or do something else?

What else would you like me to do? (or type 'terminate' to stop): Can you help me to create a documentation for this file
Executing: create_documentation with args {'file_name': 'sample_data/agent_example.py'}
Result: {'result': "Successfully created documentation file 'agent_example_documentation.txt' in directory '.'. The documentation includes code analysis, structure overview, and complete source code."}
Response: The documentation for the file `agent_example.py` has been successfully created as `agent_example_documentation.txt` in the current directory. It includes 